# Title: Discover Neighborhood with Your Favorite Restaurant

## 1. Introduction / Business Problem

Ofen time, you get tired of the restaurants in your neighborhood. You want to try other restaurants, but you don't have a certain restaurant to go to. Where should you start? In this report, neighborhoods in jersey city were clustered based on the restaurant-style, this can give you a guide on which neighborhoods to go that most fit your style. Don't hesitate and just follow the guidance.

## 2. Data Requirement and Collection

The development of the work is based on two data-sets. The first dataset is the information of all neighborhoods in jersey city including postal code, name, latitude, and longitude. This dataset will be generated by scraping from Wikipedia. The second dataset is about the number of restaurants with different styles in the neighborhood. This dataset is mainly used for clustering. It will be generated using the Foursquare API.